Dashboard Objective: Develop a Crypto Ticker dashboard that either allows the user to enter a ticker symbol into an input box, or to select item(s) from a dropdown list, and uses pandas_datareader to look up and display stock data on a graph.

# EXPAND CRYPTOCURRENCY SYMBOL INPUT TO PERMIT MULTIPLE CURRENCY SELECTION

In [ ]:
# Import dependencies

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas_datareader.data as web # requires v0.6.0 or later
from datetime import datetime
import pandas as pd

# Import API key from a file
from config import api_key

In [ ]:
# Launch Dash application

app = dash.Dash()

In [ ]:
# read a .csv file, make a dataframe, and build a list of Dropdown options

crypto = pd.read_csv('../data/All_top_coins-updated-21.csv')
#crypto.set_index('Symbol', inplace=True)
options = []
for tic in crypto.index:
    options.append({'label':'{} {}'.format(tic,crypto.loc[tic]['Currency Name']), 'value':tic})

In [ ]:
# create a Div to contain basic headers, an input box, and graph
app.layout = html.Div([
    html.H1('Crypto Crystal Ball Dashboard'),
    html.Div([
        # add styles to enlarge the input box and make room for DatePickerRange
        html.H3('Select Crypto Symbols:', style={'paddingRight':'30px'}),
        # replace dcc.Input with dcc.Options, set options=options
        dcc.Dropdown(
            # add an ID to the input box
            id='my_ticker_symbol',
            options=options,
            # sets a default value
            value=['BTC'], 
            multi=True
        )
    # widen the Div to fit multiple inputs
    ], style={'display':'inline-block', 'verticalAlign':'top', 'width':'30%'}),
    # add a Div to contain the DatePickerRange
    html.Div([
        html.H3('Select start and end dates:'),
        dcc.DatePickerRange(
            id='my_date_picker',
            min_date_allowed=datetime(2016, 3, 18),
            max_date_allowed=datetime.today(),
            start_date=datetime(2021, 4, 16),
            end_date=datetime.today()
        )
    ], style={'display':'inline-block'}),
    # add a Button element
    html.Div([
        html.Button(
            id='submit-button',
            n_clicks=0,
            children='Submit',
            style={'fontSize':24, 'marginLeft':'30px'}
        ),
    ], style={'display':'inline-block'}),
    dcc.Graph(
        # add an ID  to the graph
        id='my_graph',
        figure={
            'data': [
                {'x': [1,2], 'y': [3,1]}
            ]
        }
    )
])

# Add a callback function
@app.callback(
    # add inputs from the DatePickerRange component
    Output('my_graph', 'figure'),
    # add a button input, and move previous inputs to State
    [Input('submit-button', 'n_clicks')],
    [State('my_ticker_symbol', 'value'),
    State('my_date_picker', 'start_date'),
    State('my_date_picker', 'end_date')])

# pass n_clicks into the output function
def update_graph(n_clicks, stock_ticker, start_date, end_date):
    # Use datareader and datetime to define a DataFrame
    start = datetime.strptime(start_date[:10], '%Y-%m-%d')
    end = datetime.strptime(end_date[:10], '%Y-%m-%d')
    # since stock_ticker is now a list of symbols, create a list of traces
    traces = []
    for tic in stock_ticker:
        df = web.DataReader(tic,'iex',start,end)
        traces.append({'x':df.index, 'y': df.close, 'name':tic})
# Change the output data
    fig = {
        # set data equal to traces
        'data': traces,
        # use string formatting to include all symbols in the chart title
        'layout': {'title':', '.join(stock_ticker)+' Closing Prices'}
    }
    return fig


# Add the server clause
if __name__ == '__main__':
    app.run_server()